In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [3]:
loader = PyPDFDirectoryLoader("./us_census/")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_documents = text_splitter.split_documents(documents)
len(final_documents[0].page_content)

962

In [4]:
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device":"cpu"},
    encode_kwargs={"normalize_embeddings":True}
)



C:\Users\Administrator\AppData\Local\Temp\ipykernel_27380\86108442.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  huggingface_embeddings = HuggingFaceBgeEmbeddings(
c:\Users\Administrator\Desktop\langchain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import numpy as np

print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)))
print(np.array(huggingface_embeddings.embed_query(final_documents[0].page_content)).shape)

[-2.49750484e-02  7.48805422e-03  6.07666112e-02  2.87433490e-02
  7.38344118e-02  1.22540496e-01 -1.49952564e-02 -7.34630832e-03
 -1.00141093e-01 -1.65566721e-03 -1.06491121e-02  9.63609368e-02
 -2.55548488e-02 -1.02058545e-01  1.60376336e-02  1.36243049e-02
  1.47681823e-02 -3.22496146e-02 -2.84116510e-02  8.82342458e-02
 -3.01535465e-02  2.01462489e-02 -5.24294935e-02 -3.65701946e-03
  1.23102628e-02  5.61481016e-03  4.09074463e-02 -1.13210361e-03
 -9.47220251e-03  6.47103786e-02  8.98134261e-02 -5.35431830e-03
  2.55072415e-02  1.17485775e-02  2.94315815e-02 -4.34862264e-02
 -3.71153317e-02  2.39203293e-02 -9.60926861e-02  1.52798528e-02
 -3.21496315e-02 -4.64300998e-02 -8.10123980e-02  1.04659766e-01
  2.16493234e-02 -1.19761471e-03 -4.37056534e-02  8.71710852e-02
 -1.37321325e-02  5.47637045e-02  7.84174656e-04  3.08593344e-02
  3.65715921e-02  1.41634382e-02  1.16747720e-02 -3.02507840e-02
 -3.20119001e-02 -2.99772639e-02 -2.04795245e-02 -1.95120345e-03
 -1.50643867e-02 -2.05804

In [6]:
vectorstores = FAISS.from_documents(final_documents[:120],huggingface_embeddings)

In [7]:
query = "What is Health Insurance Coverage?"
response = vectorstores.similarity_search(query)
print(response[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [8]:
retriever = vectorstores.as_retriever(search_type="similarity",search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025B27AD90D0> search_kwargs={'k': 3}


In [9]:
import os
from dotenv import load_dotenv


load_dotenv()
os.environ["HUGGINGFACEHUB_API_TOKEN"]=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [36]:
from langchain_community.llms import HuggingFaceHub

hf = HuggingFaceHub(repo_id="gpt2",
model_kwargs={"temperature":0.1,"max_length":500},
 task="text-generation" 
)


In [37]:
query = "What is health insurance coverage?"
response = hf.invoke(query)
response

'What is health insurance coverage?\n\nHealth insurance is a form of insurance that is offered to all Americans. It is a form of insurance that is offered to all Americans. It is a form of insurance that is offered to all Americans. It is a form of insurance that is offered to all Americans. It is a form of insurance that is offered to all Americans. It is a form of insurance that is offered to all Americans. It is a form of insurance that is offered to all Americans. It is a form of insurance that is offered to all Americans. It is a form of insurance that is offered to all Americans. It is a form of insurance that is offered to all Americans. It is a form of insurance that is offered to all Americans. It is a form of insurance that is offered to all Americans. It is a form of insurance that is offered to all Americans. It is a form of insurance that is offered to all Americans. It is a form of insurance that is offered to all Americans. It is a form of insurance that is offered to al

In [ ]:
# #Hugging Face models can be run locally through the HuggingFacePipeline class.
# from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

# hf = HuggingFacePipeline.from_model_id(
#     model_id="gpt2",
#     task="text-generation",
#     pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
# )

# llm = hf 
# llm.invoke(query)

In [38]:

prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
 """

In [39]:
prompt  = PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [40]:
retrievalQA=RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [41]:

query="""DIFFERENCES IN THE
UNINSURED RATE BY STATE
IN 2022"""

In [42]:
result = retrievalQA.invoke({"query":query})


In [44]:
print(result['result'])


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

8 U.S. Census Bureau
which was the state’s uninsured 
rate in 2019, the year before it 
expanded Medicaid eligibility.22
Declines in the uninsured rate in 
the majority of these states were 
related to changes in their public 
and private coverage rates. For 
seven of the states with lower 
uninsured rates in 2022, the dif-
ference was driven by increases 
in private coverage. These states 
were Florida, Kansas, Mississippi, 
North Carolina, Ohio, South 
Carolina, and Texas.
For seven states, the uninsured 
rate decrease was related to 
increases in public coverage with 
no corresponding change in the 
level of private coverage. These 
states were Alabama, California, 
Georgia, Illinois, Indiana, Michigan, 
and Oklahoma. In three states 
(Missouri, New York, and Virginia), 
it was shifts in coverage from pri-
vate to public that contributed to 
the decline in th